<a href="https://colab.research.google.com/github/Prathudi/Fake-News-Prediction-/blob/main/FAKE_NEWS_DETECTION_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [13]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')


In [14]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [15]:
data_fake["class"] = 1
data_true["class"] = 0


In [16]:
data_fake.shape , data_true.shape

((23481, 5), (21417, 5))

In [37]:
for i in range(23480, 23470, -1):
    if i in data_fake.index:
        data_fake.drop([i], axis=0, inplace=True)
for i in range(21416, 21406, -1):
    if i in data_true.index:
        data_true.drop([i], axis=0, inplace=True)

In [41]:
data_fake.shape , data_true.shape

((23471, 5), (21407, 5))

In [42]:
data_fake_manual_testing['class']= 1
data_true_manual_testing['class']= 0

<ipython-input-42-709caa4d93b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class']= 1
<ipython-input-42-709caa4d93b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class']= 0


In [43]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23461,REPORT: ‘Federal Government Escalated the Viol...,KILLED: Rancher and protest spokesman Robert ...,Middle-east,"January 28, 2016",1
23462,"BOILER ROOM – Oregon Standoff, Cuddle Parties,...",Tune in to the Alternate Current Radio Network...,Middle-east,"January 28, 2016",1
23463,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",Patrick Henningsen 21st Century Wire UPDATE: 1...,Middle-east,"January 27, 2016",1
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Dril...,Episode #119 of SUNDAY WIRE SHOW finally resum...,Middle-east,"January 24, 2016",1
23465,‘There’ll be boots on the ground’: US making n...,21st Century Wire says Various parties in Wash...,Middle-east,"January 23, 2016",1
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",1
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",1
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",1
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",1
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",1


In [44]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",0
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",0
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",0
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",0
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",0
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",0


In [45]:
data_merge = pd.concat([data_fake , data_true] , axis = 0)
data_merge.head(10)


,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",1
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",1
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",1
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",1
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",1


In [46]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [47]:
data = data_merge.drop(['title', 'subject' , 'date'] , axis = 1)

In [48]:
data.isnull().sum()

,0
text,0
class,0


In [49]:
data = data.sample(frac = 1)

In [50]:
data.head()

,text,class
8459,Donald Trump s ridiculous stunt on Thursday ha...,1
5729,(Reuters) - Scientists may steal a page from t...,0
17766,Beyonce made an attempt to glorify the violent...,1
7864,"On Wednesday, President Obama signed a bill th...",1
12456,WASHINGTON (Reuters) - Secretary of State Rex ...,0


In [51]:
data.reset_index(inplace = True)
data.drop(['index'], axis =1 , inplace= True)


In [52]:
data.columns

Index(['text', 'class'], dtype='object')

In [53]:
data.head()

,text,class
0,Donald Trump s ridiculous stunt on Thursday ha...,1
1,(Reuters) - Scientists may steal a page from t...,0
2,Beyonce made an attempt to glorify the violent...,1
3,"On Wednesday, President Obama signed a bill th...",1
4,WASHINGTON (Reuters) - Secretary of State Rex ...,0


In [54]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W" , " ", text)
  text = re.sub('https?://\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ',text)
  text = re.sub('\n' , '', text)
  text = re.sub('\w*\d\w*' , ' ' , text)
  return text



In [55]:
data['text'] = data['text'].apply(wordopt)

In [56]:
x = data['text']
y = data['class']

In [57]:
x_train , x_test , y_train , y_test = train_test_split(x,y, test_size = 0.25)

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [61]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [62]:
pred_lr = LR.predict(xv_test)

In [63]:
LR.score(xv_test , y_test)

0.9887700534759358

In [64]:
print(classification_report(y_test , pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5304
           1       0.99      0.99      0.99      5916

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [65]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [66]:
pred_dt = DT.predict(xv_test)

In [67]:
DT.score(xv_test , y_test)

0.9944741532976827

In [68]:
print(classification_report(y_test , pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5304
           1       0.99      0.99      0.99      5916

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [69]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train , y_train)

GradientBoostingClassifier(random_state=0)

In [70]:
pred_gb = GB.predict(xv_test)

In [71]:
GB.score(xv_test , y_test)

0.9959001782531194

In [72]:
print(classification_report(y_test , pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5304
           1       0.99      0.99      0.99      5916

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [73]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train , y_train)

RandomForestClassifier(random_state=0)

In [74]:
pred_rf = RF.predict(xv_test)

In [75]:
print(classification_report(y_test , pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5304
           1       0.99      0.99      0.99      5916

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [106]:
# def output_lable(n):
#   if n==1:
#     return "Not A Fake News"
#   elif n==0:
#     return "Fake News"

# def manual_testing(news):
#     testing_news = {"text" :[news]}
#     new_def_test = pd.DataFrame(testing_news)
#     new_def_test["text"] = new_def_test["text"].apply(wordopt)
#     new_x_test = new_def_test["text"]
#     new_xv_test = vectorization.transform(new_x_test)
#     pred_LR = LR.predict(new_xv_test)
#     pred_DT = DT.predict(new_xv_test)
#     pred_GB = GB.predict(new_xv_test)
#     pred_RF = RF.predict(new_xv_test)

def output_lable(n):
  if n==0:
    return "Not A Fake News"
  elif n==1:
    return "Fake News"

def manual_testing(news):
    testing_news = {"text" :[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}" .format(output_lable(pred_LR[0]),
                                                                                                             output_lable(pred_DT[0]),
                                                                                                             output_lable(pred_GB[0]),
                                                                                                             output_lable(pred_RF[0])))


In [93]:
news = str(input())
manual_testing(news)

In a significant development, Pune has reported zero new cases of Guillain-Barré Syndrome (GBS) for the first time in over a month. This comes after a period of heightened concern due to a surge in GBS cases in the region. Health authorities have been closely monitoring the situation, and the recent data suggests a positive trend towards containment of the syndrome.


LR Prediction: Not A Fake News 
DT Prediction: Not A Fake News 
GB Prediction: Not A Fake News 
RF Prediction: Not A Fake News


In [109]:
news = "Government to deposit ₹1 lakh in every citizen’s bank account by next week."
manual_testing(news)




LR Prediction: Fake News 
DT Prediction: Fake News 
GB Prediction: Fake News 
RF Prediction: Fake News
